# 3_DecisionTree_Diabetes

**Proyecto:** MLY0100 — Predicción de Riesgo de Diabetes  
**Modelo:** Árbol de Decisión (Decision Tree Classifier)  
**Autor:** Antonio Sepúlveda  
**Fecha:** 2025

## 1. Conexión con Kedro y carga del dataset

En este notebook se implementa un modelo de **Árbol de Decisión** para clasificar pacientes según la variable objetivo **Outcome**:
- `0` → No Diabetes  
- `1` → Diabetes

Se utilizarán los datos limpios generados por el pipeline de Kedro (`diabetes_cleaned`).

In [ ]:
%load_ext kedro.ipython
%reload_kedro

# Listar datasets disponibles en el catálogo
catalog.list()

In [ ]:
import pandas as pd

# Cargar dataset limpio desde Kedro
df_diabetes = catalog.load('diabetes_cleaned')

print(df_diabetes.shape)
df_diabetes.head()

### 1.1 Distribución de la variable objetivo (Outcome)

Revisamos el equilibrio de clases para entender si el problema está balanceado o desbalanceado.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style='whitegrid')

plt.figure(figsize=(5,4))
sns.countplot(x='Outcome', data=df_diabetes)
plt.title('Distribución de la variable Outcome')
plt.xlabel('Outcome (0 = No Diabetes, 1 = Diabetes)')
plt.ylabel('Cantidad de pacientes')
plt.show()

df_diabetes['Outcome'].value_counts(normalize=True)

## 2. Importaciones para modelado y métricas

Se importan las librerías necesarias para entrenamiento, evaluación y visualización del Árbol de Decisión.

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score,
    f1_score, classification_report, roc_curve, auc,
    precision_recall_curve, average_precision_score
)

## 3. Selección de características

Utilizaremos todas las variables numéricas del dataset como características (`X`) y la variable **Outcome** como objetivo (`y`).

In [ ]:
# X = todas las columnas excepto Outcome
X = df_diabetes.drop('Outcome', axis=1)
y = df_diabetes['Outcome']

print('Dimensiones de X:', X.shape)
print('Dimensiones de y:', y.shape)
X.head()

## 4. División de datos en entrenamiento y prueba

Se divide el conjunto en:
- 80% para **entrenamiento**
- 20% para **prueba**

Se utiliza `stratify=y` para conservar la proporción de clases.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print('X_train:', X_train.shape)
print('X_test :', X_test.shape)

## 5. Entrenamiento del modelo Árbol de Decisión (versión base)

Primero se entrena un modelo base con hiperparámetros simples para entender el comportamiento inicial.

In [ ]:
dt_base = DecisionTreeClassifier(
    random_state=42,
    max_depth=4,      # profundidad controlada para evitar sobreajuste
    criterion='gini'  # criterio estándar
)

dt_base.fit(X_train, y_train)

y_pred_base = dt_base.predict(X_test)
y_proba_base = dt_base.predict_proba(X_test)[:, 1]

## 6. Métricas de evaluación — Modelo base

Se calculan las métricas principales:
- Accuracy  
- Precision  
- Recall (Sensitivity)  
- F1-score

In [ ]:
acc_base = accuracy_score(y_test, y_pred_base)
prec_base = precision_score(y_test, y_pred_base, zero_division=0)
rec_base = recall_score(y_test, y_pred_base, zero_division=0)
f1_base = f1_score(y_test, y_pred_base, zero_division=0)

print('🔍 Métricas modelo base (Árbol de Decisión):\n')
print(f"Accuracy : {acc_base:.4f}")
print(f"Precision: {prec_base:.4f}")
print(f"Recall   : {rec_base:.4f}")
print(f"F1-score : {f1_base:.4f}\n")

print('📋 Classification Report:\n')
print(classification_report(y_test, y_pred_base, zero_division=0))

### 6.1 Matriz de confusión — Modelo base

Se visualiza la matriz de confusión para analizar los aciertos y errores del modelo.

In [ ]:
cm_base = confusion_matrix(y_test, y_pred_base)
cm_base

In [ ]:
plt.figure(figsize=(5,4))
sns.heatmap(cm_base, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de confusión — Árbol de Decisión (Base)')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

### 6.2 Sensitivity y Specificity — Modelo base

- **Sensitivity (Recall de la clase positiva)** = TP / (TP + FN)  
- **Specificity** = TN / (TN + FP)

In [ ]:
tn, fp, fn, tp = cm_base.ravel()

sensitivity_base = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity_base = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"Sensitivity (Recall, clase 1): {sensitivity_base:.4f}")
print(f"Specificity (clase 0)        : {specificity_base:.4f}")

## 7. Curva ROC y Curva Precisión-Recall — Modelo base

Se calculan y visualizan las curvas **ROC** y **Precisión-Recall** para evaluar el rendimiento del modelo en términos de umbrales de decisión.

In [ ]:
# --- Curva ROC ---
fpr_base, tpr_base, _ = roc_curve(y_test, y_proba_base)
roc_auc_base = auc(fpr_base, tpr_base)

plt.figure(figsize=(6,5))
plt.plot(fpr_base, tpr_base, color='darkorange', lw=2,
         label=f'ROC curve (AUC = {roc_auc_base:.2f})')
plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC — Árbol de Decisión (Base)')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# --- Curva Precisión-Recall ---
precision_base, recall_base, _ = precision_recall_curve(y_test, y_proba_base)
ap_base = average_precision_score(y_test, y_proba_base)

plt.figure(figsize=(6,5))
plt.plot(recall_base, precision_base, color='blue', lw=2,
         label=f'PR curve (AP = {ap_base:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall — Árbol de Decisión (Base)')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()

## 8. Importancia de características — Modelo base

Los Árboles de Decisión permiten interpretar la importancia relativa de cada variable para la clasificación.

In [ ]:
feature_importances = pd.Series(dt_base.feature_importances_, index=X.columns)
feature_importances = feature_importances.sort_values(ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x=feature_importances.values, y=feature_importances.index)
plt.title('Importancia de características — Árbol de Decisión (Base)')
plt.xlabel('Importancia')
plt.ylabel('Variables')
plt.tight_layout()
plt.show()

feature_importances

## 9. Visualización del Árbol de Decisión (opcional)

A continuación se visualiza el árbol entrenado (cuando la profundidad no es muy grande). Esto permite interpretar reglas de decisión.

In [ ]:
plt.figure(figsize=(16,10))
plot_tree(
    dt_base,
    feature_names=X.columns,
    class_names=['No Diabetes', 'Diabetes'],
    filled=True,
    rounded=True,
    max_depth=3  # limitar profundidad en la visualización
)
plt.title('Árbol de Decisión — Visualización Parcial (max_depth=3)')
plt.show()

## 10. Búsqueda de hiperparámetros con GridSearchCV

Se realiza una búsqueda de hiperparámetros para mejorar el rendimiento del modelo utilizando **GridSearchCV**.

Hiperparámetros a explorar:
- `max_depth`
- `min_samples_split`
- `min_samples_leaf`
- `criterion`

In [ ]:
param_grid = {
    'max_depth': [3, 4, 5, 6, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

dt = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print('Mejores parámetros:', grid_search.best_params_)
print('Mejor accuracy (CV):', round(grid_search.best_score_, 4))

## 11. Evaluación del mejor modelo (GridSearchCV)

Se evalúa el mejor árbol encontrado por GridSearchCV sobre el conjunto de prueba.

In [ ]:
best_dt = grid_search.best_estimator_

y_pred_best = best_dt.predict(X_test)
y_proba_best = best_dt.predict_proba(X_test)[:, 1]

acc_best = accuracy_score(y_test, y_pred_best)
prec_best = precision_score(y_test, y_pred_best, zero_division=0)
rec_best = recall_score(y_test, y_pred_best, zero_division=0)
f1_best = f1_score(y_test, y_pred_best, zero_division=0)

print('🔍 Métricas mejor modelo (GridSearchCV — Árbol de Decisión):\n')
print(f"Accuracy : {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall   : {rec_best:.4f}")
print(f"F1-score : {f1_best:.4f}\n")

print('📋 Classification Report (Best Model):\n')
print(classification_report(y_test, y_pred_best, zero_division=0))

### 11.1 Matriz de confusión — Mejor modelo


In [ ]:
cm_best = confusion_matrix(y_test, y_pred_best)
plt.figure(figsize=(5,4))
sns.heatmap(cm_best, annot=True, fmt='d', cmap='Greens')
plt.title('Matriz de confusión — Árbol de Decisión (Best Model)')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

cm_best

### 11.2 Sensitivity y Specificity — Mejor modelo


In [ ]:
tn_b, fp_b, fn_b, tp_b = cm_best.ravel()

sensitivity_best = tp_b / (tp_b + fn_b) if (tp_b + fn_b) > 0 else 0
specificity_best = tn_b / (tn_b + fp_b) if (tn_b + fp_b) > 0 else 0

print(f"Sensitivity (Best Model): {sensitivity_best:.4f}")
print(f"Specificity (Best Model): {specificity_best:.4f}")

### 11.3 Curva ROC y Curva PR — Mejor modelo


In [ ]:
# --- Curva ROC (Best Model) ---
fpr_b, tpr_b, _ = roc_curve(y_test, y_proba_best)
roc_auc_b = auc(fpr_b, tpr_b)

plt.figure(figsize=(6,5))
plt.plot(fpr_b, tpr_b, color='darkorange', lw=2,
         label=f'ROC curve (AUC = {roc_auc_b:.2f})')
plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC — Árbol de Decisión (Best Model)')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# --- Curva Precision-Recall (Best Model) ---
precision_b, recall_b, _ = precision_recall_curve(y_test, y_proba_best)
ap_b = average_precision_score(y_test, y_proba_best)

plt.figure(figsize=(6,5))
plt.plot(recall_b, precision_b, color='blue', lw=2,
         label=f'PR curve (AP = {ap_b:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall — Árbol de Decisión (Best Model)')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()